En esta segunda prueba vamos a incorporar solo las nuevas variables donde hemos observado que tienen relación con ser fumador que tienen un pvalue menor a 0.05 y vamos a crear algunas nuevas. Después, vamos a generar características polinómicas y a escalar los datos. 

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [34]:
df=pd.read_csv("train.csv")

In [35]:
df.drop(columns="id", inplace=True)

### Crear funciones para añadir variables al df_train y df_test

Hemos observado en las gráficas del archivo anterior que los fumadores son más altos que los no fumadores, por ello hemos dividido la altura por tramos también. A partir del tramo donde hemos observado más fumadores que no fumadores. También hemos observado que a partir de los 70 kg hay más fumadores, por ello dividimos el peso por kgs también. 
Por último, eliminamos algunas columnas que no nos aportan información para predecir nuestra Y.

In [36]:
def transforma(df):
    df["IMC"] = df["weight(kg)"] / ((df['height(cm)'] / 100) ** 2)
    df['weight_to_hip_ratio'] = df['weight(kg)'] / df['waist(cm)']
    df['age_category'] = pd.cut(df['age'], bins=[ 18, 35, 50, 100], labels=[ '18-35', '35-50', '50+'])
    df['Colesterol_LDL'] = df['LDL'] / df['Cholesterol']
    df['Media_vista'] = (df['eyesight(left)'] + df['eyesight(right)'])
    df['Vista_tramos'] = pd.cut(df['Media_vista'], bins=[0, 2.7, 4, 20], labels=[ 'Baja', 'Media', 'Alta'])
    df['altura_tramos'] = pd.cut(df['height(cm)'], bins=[120, 160, 190], labels=[ 'Bajo', 'Alto'])
    df['peso_tramos'] = pd.cut(df['weight(kg)'], bins=[20, 70, 140], labels=[ 'Bajo', 'Alto'])

In [37]:
transforma(df)

In [38]:
df =pd.get_dummies(df, columns=['age_category'], drop_first=True)

In [39]:
def presión_arterial_alta(systolic, relaxation):
    if systolic >= 130 and relaxation > 80:
        return 1
    else:
        return 0

In [40]:
df['Presión arterial'] = df.apply(lambda row: presión_arterial_alta(row['systolic'], row['relaxation']), axis=1)

In [41]:
df =pd.get_dummies(df, columns=['Vista_tramos'], drop_first=True)

In [42]:
df =pd.get_dummies(df, columns=['altura_tramos'], drop_first=True)

In [43]:
df =pd.get_dummies(df, columns=['peso_tramos'], drop_first=True)

In [44]:
df.drop(columns="Media_vista", inplace=True)

In [45]:
df.drop(columns="eyesight(left)", inplace=True)

### Hacemos los mismos cambios en el df_test

In [46]:
df_test=pd.read_csv("test.csv")
X_test=df_test.drop(columns="id")

In [47]:
transforma(X_test)

In [48]:
X_test =pd.get_dummies(X_test, columns=['age_category'], drop_first=True)

In [49]:
X_test['Presión arterial'] = X_test.apply(lambda row: presión_arterial_alta(row['systolic'], row['relaxation']), axis=1)

In [50]:
X_test =pd.get_dummies(X_test, columns=['Vista_tramos'], drop_first=True)

In [51]:
X_test =pd.get_dummies(X_test, columns=['altura_tramos'], drop_first=True)

In [52]:
X_test =pd.get_dummies(X_test, columns=['peso_tramos'], drop_first=True)

In [53]:
X_test.drop(columns="Media_vista", inplace=True)

In [54]:
X_test.drop(columns="eyesight(left)", inplace=True)

### Generación de características polinómicas

Lo hacemos para reducir la asimetría en la distribución de datos y comprimir los valores más grandes.

In [57]:
X=df.drop(columns="smoking")
Y=df.smoking

In [58]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree= 2, include_bias=False, interaction_only= True) #include_bias= no quiero que me haga la constante
poly.fit(X)
X=pd.DataFrame(poly.transform(X), columns= poly.get_feature_names_out())

### Escalamos los datos

In [59]:
scaler=StandardScaler()
scaler.fit(X)
X=pd.DataFrame(scaler.transform(X),columns=X.columns)

### Creamos el modelo de regresión logística

In [60]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=10000, C=0.1)
logistic_model.fit(X, Y)

LogisticRegression(C=0.1, max_iter=10000)

### Generación de características polinómicas para el X_test

In [61]:
X_test=pd.DataFrame(poly.transform(X_test), columns= poly.get_feature_names_out())

### Escalo los datos del df_test

In [62]:
X_test=pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

### Obtengo las probabilidades

In [63]:
y_probs_test=logistic_model.predict_proba(X_test)[:,1]

### Creo un nuevo df

In [64]:
submission1 =pd.DataFrame({"id":df_test.id,"smoking":y_probs_test})

In [66]:
submission1.to_csv('predicciones_fumar.csv', index=False)

In [65]:
submission1

,id,smoking
0,159256,0.534020
1,159257,0.244660
2,159258,0.394515
3,159259,0.036705
4,159260,0.650595
...,...,...
106166,265422,0.579635
106167,265423,0.705990
106168,265424,0.459917
106169,265425,0.059831
